In [2]:
import numpy as np
import open3d as o3d

def custom_draw_geometry_with_key_callback(pcd):
    
    def change_background_to_black(vis):
        opt = vis.get_render_option()
        #print(dir(opt))       
        opt.background_color = np.array([0, 0, 0])
        return False
    
    def change_background_to_white(vis):
        opt = vis.get_render_option()
        opt.background_color = np.array([255, 255, 255])
        return False
    
    def points_size_up(vis):
        opt = vis.get_render_option()
        opt.point_size = opt.point_size+0.5
        return False
    
    def points_size_down(vis):
        opt = vis.get_render_option()
        opt.point_size = opt.point_size-0.5
        return False   
    
    def do_nothing(vis):
        return False 

    key_to_callback = {}
    key_to_callback[ord("Q")] = change_background_to_black
    key_to_callback[ord("W")] = change_background_to_white
    key_to_callback[ord("1")] = points_size_down
    key_to_callback[ord("2")] = points_size_up
    key_to_callback[ord("3")] = do_nothing
    key_to_callback[ord("4")] = do_nothing
    
    o3d.visualization.draw_geometries_with_key_callbacks([pcd],
                                                         key_to_callback,
                                                         width=1000, height=600)
    
print('o3d version', o3d.__version__)

o3d version 0.11.2


### Read Data

In [5]:
human_pc = np.load('./SUOD/SUOD/points_val/18.npy')
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(human_pc)
print(i, human_pc.shape)
#human_pc[:5]

29 (294, 3)


### Normal Estimation (требуются для 2/3 алгоритмов, но у нас их изначально нет)

- http://www.open3d.org/docs/0.7.0/tutorial/Basic/pointcloud.html#vertex-normal-estimation
- http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Normal-Estimation

In [8]:
n = np.empty((human_pc.shape[0],4))
n[:,:3] = -np.asarray(pcd.points)
n[:,3] = np.sqrt(n[:,0]**2+n[:,1]**2+n[:,2]**2)
n[:,0]/=n[:,3]
n[:,1]/=n[:,3]
n[:,2]/=n[:,3]
n[:,3] = n[:,0]**2+n[:,1]**2+n[:,2]**2

pcd.normals = o3d.utility.Vector3dVector(n[:,:3])
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 


In [24]:
#pcd.estimate_normals()
#pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#o3d.visualization.draw_geometries([pcd], point_show_normal=True)

### Alpha shapes (не требует pc.normals)

- http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Alpha-shapes

In [12]:
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, 0.18)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 
[Open3D WARNING]

### Ball pivoting (требует pc.normals)

- http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Ball-pivoting

In [66]:
#ntr_max = 0
#for i in range(20000):
#    radii = [0.00001*(i+1)]
#    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))
#    ntr_curr = int(str(rec_mesh).split()[-2])
#    if ntr_curr > ntr_max:
#        ntr_max = ntr_curr
#        print(radii, ntr_max) # --> [0.11531] 83

In [10]:
r = 0.11531
optimal_radii = [r*x for x in [1,2]]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,
                                                                           o3d.utility.DoubleVector(optimal_radii))
ntr_curr = int(str(rec_mesh).split()[-2])
print(ntr_curr)
o3d.visualization.draw_geometries([pcd, rec_mesh], mesh_show_back_face=True, point_show_normal=True)

253
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 


### Poisson surface reconstruction (требует pc.normals)

- http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Poisson-surface-reconstruction

In [11]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
#print(mesh)

o3d.visualization.draw_geometries([pcd, mesh], mesh_show_back_face=True, point_show_normal=True,
                                  width=1500, height=900)

[Open3D DEBUG] Input Points / Samples: 294 / 294
[Open3D DEBUG] #   Got kernel density: 0.00400019 (s), 114.633 (MB) / 114.633 (MB) / 245 (MB)
[Open3D DEBUG] #     Got normal field: 0.00300002 (s), 115.34 (MB) / 115.34 (MB) / 245 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.142963e-03 / 6.300310e-01
[Open3D DEBUG] #       Finalized tree: 0.0109999 (s), 120.566 (MB) / 120.566 (MB) / 245 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.036 (s), 119.746 (MB) / 120.566 (MB) / 245 (MB)
[Open3D DEBUG] #Set point constraints: 0.00200009 (s), 119.512 (MB) / 120.566 (MB) / 245 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 82678 / 67808 / 26681
[Open3D DEBUG] Memory Usage: 119.516 MB
[Open3D DEBUG] # Linear system solved: 0.0699999 (s), 123.281 (MB) / 123.281 (MB) / 245 (MB)
[Open3D DEBUG] Got average: 0.00300002 (s), 119.539 (MB) / 123.281 (MB) / 245 (MB)
[Open3D DEBUG] Iso-Value: 4.967621e-01 = 1.460480e+02 / 2.940000e+02
[Open3D DEBUG] #          Total Solve:       0.3 (s)